In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/ML/hw3-2023')

## Import packages

In [3]:
import sys
import pandas as pd

if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
# from scipy.stats import multivariate_normal
from skimage.feature import hog, local_binary_pattern

## Load data

In [4]:
# load data
data_name = "mnist"
data = np.load(f"./data/{data_name}-data-hw3.npz")
print("\nloaded %s data!" % data_name)
fields = "test_data", "training_data", "training_labels"
for field in fields:
    print(field, data[field].shape)


loaded mnist data!
test_data (10000, 1, 28, 28)
training_data (60000, 1, 28, 28)
training_labels (60000,)


## Extract features

In [5]:
def extract_lbp_features(img, lbp_radius=1, lbp_point=8):
    lbp = local_binary_pattern(img, lbp_point, lbp_radius, 'default')
    max_bins = int(lbp.max() + 1)
    # hist size:256
    hist, _ = np.histogram(lbp, density=True, bins=max_bins, range=(0, max_bins))
    return hist

def extract_hog_features(img):
    return hog(img)


def extract_features(data, width=28, height=28, is_grey_image=True):
    image_descriptors = []
    arr = np.array(data)
    print(arr.shape)
    for x in data:
      if is_grey_image:
        x = x.reshape(width, height)
        # normalize
        x /= np.linalg.norm(x)
        fd = []
      fd = np.append(fd, extract_hog_features(x))
      fd = np.append(fd, extract_lbp_features(x))
      fd = np.append(fd, x)
      image_descriptors.append(fd)
    return image_descriptors

In [6]:
# extract features from raw pixels
training_data = extract_features(data["training_data"])
test_data = extract_features(data["test_data"])

(60000, 1, 28, 28)
(10000, 1, 28, 28)


In [7]:
data_flat = np.array(training_data)
labels = data["training_labels"].reshape(len(data["training_labels"]), -1)
dataset = np.concatenate([labels, data_flat], axis=-1)
print(dataset.shape)

(60000, 1122)


In [8]:
# split data
np.random.seed(113)
np.random.shuffle(dataset)
val_set = dataset[0:10000]
train_set = dataset[10000:]

## Define model

In [9]:
# define GDA(LDA/QDA) model
class GDA:
  def __init__(self, mode="lda"):
    self.mode = mode
    self.mus = []
    self.covs = []
    self.prior_probs = []
    self.cov_invs, self.cov_logdets = [], []

  def train(self, train_set):
    """
    Calculate means and covariance matrixes given x and y
    (If mode set to lda, calculate cov's average.)
    :param train_set: for a training pts, first dim is its label 
    """
    train_set = np.array(train_set)
    total = train_set.shape[0]
    for label in range(10):
      print("Computing mean and covariance matrix for class {}".format(label))
      data_tmp = train_set[np.where(train_set[:, 0].astype(np.int64) == label)][:, 1:]
      mu = np.mean(data_tmp, axis=0)
      self.mus.append(mu)
      cov = (data_tmp - mu).T @ (data_tmp - mu) / data_tmp.shape[0]
      self.covs.append(cov)
      prior_prob = data_tmp.shape[0]/total
      self.prior_probs.append(prior_prob)
    if self.mode=="lda":
      self.cov = np.average(self.covs, axis=0)
    else:
      self.covs = np.array(self.covs)
    self.get_invs_dets()
      

  def get_invs_dets(self):
    if self.mode=="lda":
      tmp = abs(self.cov)
      tmp = tmp[np.nonzero(tmp)]
      minval = np.min(tmp)
      dim = self.cov.shape[0]
      new_cov = self.cov + np.eye(dim) * minval * 0.001
      self.cov_inv = np.linalg.inv(new_cov)
      sign, cov_logdet = np.linalg.slogdet(new_cov)
      assert(sign!=0)
      self.cov_logdet = sign * cov_logdet
    else:
      tmp = abs(self.covs)
      tmp = tmp[np.nonzero(tmp)]
      minval = np.min(tmp)
      dim = self.covs.shape[1]
      print(minval)
      for cov in self.covs:
        new_cov = cov + np.eye(dim) * minval * 0.001
        cov_inv = np.linalg.inv(new_cov)
        sign, cov_logdet = np.linalg.slogdet(new_cov)
        assert(sign!=0)
        self.cov_invs.append(cov_inv)
        self.cov_logdets.append(sign * cov_logdet)


  def calc_probs(self, x):
    """
    For a given input x, return its probabilities of belonging to each class
    """
    probs = np.zeros(10,)
    for i in range(10):
      if self.mode == "lda":
        # extremely slow when using logpdf directly
        # probs[i] = multivariate_normal.logpdf(x, self.mus[i], self.cov, True)
        bias = (x-self.mus[i]).reshape((-1, 1))
        probs[i] = - bias.T @ self.cov_inv @ bias/2 - self.cov_logdet/2 + np.log(self.prior_probs[i])
      else:
        bias = (x-self.mus[i]).reshape((-1, 1))
        probs[i] = - bias.T @ self.cov_invs[i] @ bias/2 - self.cov_logdets[i]/2 + np.log(self.prior_probs[i])
        # probs[i] = multivariate_normal.logpdf(x, self.mus[i], self.covs[i], True) + np.log(self.prior_probs[i])
    return probs

  def predict(self, datas):
    predict_labels = []
    datas = np.array(datas)
    for i, data in enumerate(datas):
      i = (i+1)/datas.shape[0]
      print('\rPredicting：{}{:.2f}%'.format('▉'*int(i*50),(i*100)), end='')
      probs = self.calc_probs(data)
      label = np.argmax(probs)
      predict_labels.append(label)
    return np.array(predict_labels)

  def eval(self, val_set):
    print("Evaluating model...")
    labels = val_set[:, 0]
    datas = val_set[:, 1:]
    predict_labels = self.predict(datas)
    acc = accuracy_score(labels, predict_labels)
    err_digitwise = []
    for i in range(10):
      total = np.sum(labels==i)
      cnt = np.sum(predict_labels[np.where(labels==i)]!=i)
      err_digitwise.append(cnt/total)
    return acc, err_digitwise

## Train model

In [10]:
model = GDA("lda")
model.train(train_set)
# calculate overall error rate
total_acc, lda_err_digitwise = model.eval(val_set)
print(f"\nAcc: {total_acc*100}%")

Computing mean and covariance matrix for class 0
Computing mean and covariance matrix for class 1
Computing mean and covariance matrix for class 2
Computing mean and covariance matrix for class 3
Computing mean and covariance matrix for class 4
Computing mean and covariance matrix for class 5
Computing mean and covariance matrix for class 6
Computing mean and covariance matrix for class 7
Computing mean and covariance matrix for class 8
Computing mean and covariance matrix for class 9
Evaluating model...
Predicting：▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉100.00%
Acc: 96.03%


In [11]:
model = GDA("lda")
model.train(dataset)

Computing mean and covariance matrix for class 0
Computing mean and covariance matrix for class 1
Computing mean and covariance matrix for class 2
Computing mean and covariance matrix for class 3
Computing mean and covariance matrix for class 4
Computing mean and covariance matrix for class 5
Computing mean and covariance matrix for class 6
Computing mean and covariance matrix for class 7
Computing mean and covariance matrix for class 8
Computing mean and covariance matrix for class 9


In [13]:
# A code snippet to help you save your results into a kaggle accepted csv
def results_to_csv(y_test, name):
    y_test = y_test.astype(int)
    df = pd.DataFrame({'Category': y_test})
    df.index += 1 # Ensures that the index starts at 1
    df.to_csv(name+'.csv', index_label='Id')

In [14]:
labels = model.predict(np.array(test_data))
results_to_csv(labels, "lda_submission_mnist")

Predicting：▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉100.00%